In [39]:
import pandas as pd
from datetime import timedelta

In [40]:
# Reading dataframes

#Match Report
df = pd.read_csv('data/campeonato-brasileiro-full.csv')
#Filtering only matches envolving Corinthians
df = df.loc[(df['mandante'] == 'Corinthians') |  (df['visitante'] == 'Corinthians') ] 

# Reading Album Realeses report
df_album = pd.read_csv('data/album-releases.csv')

In [41]:
def check_result(winner):
    if winner== 'Corinthians':
        return "W" #Win
    if winner== '-':
        return "D" #Draw
    else:
        return "L" #Loss
df['result'] = df['vencedor'].apply(check_result)

In [42]:
def get_closest_date(album_release_date):
    try:
        next_game_date = min(df.loc[df['data'] > album_release_date].data)
    except:
        next_game_date = None
    return next_game_date


In [43]:
# Formating Match Report(df) 'data' field - Match date field

df['data'] = pd.to_datetime(df['data'], format = '%d/%m/%Y', errors='ignore')

## Formating Album Releases table(df_album)  'date' field - Album realease date

df_album['date'] = pd.to_datetime(df_album['date'], format = '%m/%d/%Y', errors='ignore')

# Creating a function to extract the next match after an album releases.
def get_closest_date(album_release_date):
    try:
        next_game_date = min(df.loc[df['data'] > album_release_date].data)
    except:
        next_game_date = None
    return next_game_date

#Applying the function and creating a new field 'clostest game date' in df_album
df_album['closest_game_date'] = df_album['date'].apply(get_closest_date)

#Formating closest_game_date
df_album['closest_game_date'] = pd.to_datetime(df_album['closest_game_date'], format = '%Y/%m/%d', errors='raise')

# Mergin the two dataframes into one 'df_merged
df_merged = pd.merge(left = df_album, right = df, left_on= 'closest_game_date', right_on= 'data')

In [44]:
# Filtering only the albums with a game within 7 days after the release
df_merged['days_between_release_and_game'] =  df_merged['closest_game_date'] - df_merged['date']
df_merged['released_within_7_days'] = df_merged['days_between_release_and_game'] <= timedelta(days = 7)
df_scopo = df_merged.loc[df_merged['released_within_7_days'] == True].reset_index(drop=True)

In [45]:
simple_pivot = pd.pivot_table(df_scopo, index= 'artist', values = 'album',columns = 'result', aggfunc='count', fill_value = 0, dropna= True)
simple_pivot['total'] = simple_pivot['W'] + simple_pivot['D'] + simple_pivot['L']
simple_pivot['victory_conversion'] = simple_pivot['W']/simple_pivot['total'] 
simple_pivot['victory_or_draw_conversion'] = (simple_pivot['W'] + simple_pivot['D'])/simple_pivot['total'] 
simple_pivot_filtered = simple_pivot.loc[(simple_pivot['total'] > 5) & (simple_pivot['victory_or_draw_conversion'] == 1)].sort_values('victory_conversion', ascending=False)